In [1]:
import os
import pandas as pd
import plotly.graph_objects as go
from plotly.offline import plot

# Set relative folder path
data_folder = "../input_files/input_single"

# List of filenames
files = [
    "Fugro_B16-PB1.xlsx",
    "Fugro_B17_PB1.xlsx",
    "Fugro_HB28_PB1.xlsx",
    "Fugro_HB29-PB1.xlsx"
]

# Load and merge Excel files
dfs = [pd.read_excel(os.path.join(data_folder, f), index_col='Time', parse_dates=True) for f in files]
merged_df = pd.concat(dfs, axis=1)
merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]  # Optional cleanup

In [2]:
daily_df = merged_df.resample('D').mean()
print(daily_df.head(50))

             B16-PB1   B17-PB1  HB28-PB1  HB29-PB1
Time                                              
2023-11-09       NaN       NaN       NaN       NaN
2023-11-10 -0.547714 -1.216231  2.281143  0.388154
2023-11-11 -0.633125 -1.225542  2.252792  0.343958
2023-11-12 -0.659958 -1.246542  2.121417  0.279000
2023-11-13 -0.665833 -1.254750  2.025833  0.242500
2023-11-14 -0.664583 -1.272458  1.936083  0.177625
2023-11-15 -0.667083 -1.280708  1.902125  0.183583
2023-11-16 -0.664750 -1.255125  1.945083  0.288250
2023-11-17 -0.658750 -1.270625  1.895708  0.182625
2023-11-18 -0.648125 -1.264208  1.881167  0.176833
2023-11-19 -0.628917 -1.219125  1.976583  0.342417
2023-11-20 -0.616292 -1.209417  2.079792  0.353417
2023-11-21 -0.609833 -1.227208  1.989708  0.298000
2023-11-22 -0.614208 -1.245458  1.900625  0.182083
2023-11-23 -0.610958 -1.251042  1.884500  0.158542
2023-11-24 -0.666208 -1.271542  1.846625  0.088375
2023-11-25 -0.709333 -1.284583  1.795083  0.024333
2023-11-26 -0.685208 -1.275125 

In [3]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 588 entries, 2023-11-09 to 2025-06-18
Freq: D
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   B16-PB1   565 non-null    float64
 1   B17-PB1   587 non-null    float64
 2   HB28-PB1  587 non-null    float64
 3   HB29-PB1  587 non-null    float64
dtypes: float64(4)
memory usage: 23.0 KB


In [4]:
# Example sensor map (X coordinates)
sensor_x_map = {
    'B16-PB1': 13,
    'HB28-PB1': 19,
    'HB29-PB1': 32,
    'B17-PB1': 35
}

# Reorder sensors by X coordinate
ordered_sensors = sorted(sensor_x_map.items(), key=lambda item: item[1])
sensor_names = [s[0] for s in ordered_sensors]
x_coords = [s[1] for s in ordered_sensors]

# Assume your dataframe is already named `df`
df = daily_df.copy()
df = df[sensor_names]  # Ensure correct order

# --- Define dam outline ---
dam_outline_x = [0, 12, 22, 27, 37]
dam_outline_y = [0, 4.3, 4.3, 1.0, 0]

# Create frames for animation
frames = []
for date, row in df.iterrows():
    if row.isnull().all():
        continue

    frames.append(go.Frame(
        data=[
            go.Scatter(
                x=x_coords,
                y=row.values,
                mode='lines+markers+text',
                text=sensor_names,
                textposition='top center',
                name='Phreatic Line',
                line=dict(color='blue', width=3),
                marker=dict(size=8)
            ),
            go.Scatter(
                x=dam_outline_x,
                y=dam_outline_y,
                mode='lines',
                line=dict(color='black', width=2),
                name='Dam Outline',
                showlegend=False
            )
        ],
        name=str(date.date())
    ))

# Initial visible frame
initial_row = df.dropna().iloc[0]

fig = go.Figure(
    data=[
        go.Scatter(
            x=x_coords,
            y=initial_row.values,
            mode='lines+markers+text',
            text=sensor_names,
            textposition='top center',
            name='Phreatic Line',
            line=dict(color='blue', width=3),
            marker=dict(size=8)
        ),
        go.Scatter(
            x=dam_outline_x,
            y=dam_outline_y,
            mode='lines',
            line=dict(color='black', width=2),
            name='Dam Outline',
            showlegend=True
        )
    ],
    layout=go.Layout(
        title="Phreatic Line Through Time with Dam Outline",
        xaxis=dict(title='Distance along Dam (m)', range=[0, 40]),
        yaxis=dict(title='Water Head (m)', range=[min(df.min().min(), 0) - 0.5, df.max().max() + 0.5]),
        updatemenus=[dict(
            type='buttons',
            buttons=[dict(label='Play', method='animate', args=[None])]
        )],
        sliders=[dict(
            steps=[
                dict(method='animate',
                     args=[[f.name], dict(mode='immediate', frame=dict(duration=300, redraw=True), transition=dict(duration=0))],
                     label=f.name)
                for f in frames
            ],
            transition=dict(duration=0),
            x=0.1,
            len=0.9
        )]
    ),
    frames=frames
)

# Export to HTML
plot(fig, filename='animated_phreatic_line_with_dam.html', auto_open=False)

'animated_phreatic_line_with_dam.html'